In [ ]:
import pandas as pd
import requests
import os
import re
import logging

logging.basicConfig(filename='error.log', level=logging.ERROR)

FORMAT = 'csv'
API_KEY = os.environ.get('cwb_key')
DATA_FILE = 'data_list.xlsx'
OUTPUT_DIR = f'data'

def process_row(row):
    data_id = row['資料集代碼']
    data_name = row['資料集名稱']
    data_tag = row['標籤']
    data_name = re.sub(r'[\\/:*?"<>|]', '', data_name)
    try:
        response = requests.get(f'https://opendata.cwb.gov.tw/api/v1/rest/datastore/{data_id}?format={FORMAT}&Authorization={API_KEY}')
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        logging.error(f'Error retrieving data for ID {data_id}: {str(e)}')
        return
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    file_path = os.path.join(OUTPUT_DIR, f'({data_tag}) {data_id}_{data_name}.{format}')
    with open(file_path, 'w') as f:
        f.write(response.text)

def main():
    df = pd.read_excel(DATA_FILE)
    df.apply(process_row, axis=1)

if __name__ == '__main__':
    main()
